In [ ]:
!pip install sentence-transformers

In [29]:
import pickle
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
import numpy as np
from sentence_transformers import SentenceTransformer, evaluation
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, InputExample, losses

# Data

In [6]:
train_file_path = "/kaggle/input/nlp-dataset/SP_train.npy"
eval_file_path = "/kaggle/input/nlp-dataset/SP_eval_data_for_practice.npy"

In [7]:
train_data = np.load(train_file_path ,allow_pickle = True)
eval_data = np.load(eval_file_path, allow_pickle = True)

In [9]:
train_questions = []
train_options =[]
train_labels = []

correct_options = []

for data in train_data:
    ques = data['question']
    train_questions.extend([ques]*4)
    correct_options.append(data['label'])
    
    for i in range(4):    
        choice = data['choice_list'][i]
        labl = data['choice_order'][i]
        train_options.append(choice)
        
        if labl==0:
            train_labels.append(1)
        else:
            train_labels.append(0)

In [10]:
with open('/kaggle/input/val-labels/list.pkl', 'rb') as f:
    eval_labels = pickle.load(f)

In [12]:
eval_questions = []
eval_options =[]

correct_options_eval = []

for data in eval_data:
    ques = data['question']
    eval_questions.extend([ques]*4)
    
    for i in range(4):    
        choice = data['choice_list'][i]
        eval_options.append(choice)
        

p = [eval_labels[i:i+4] for i in range(0, len(eval_labels), 4)]
for i in p:
    correct_options_eval.append(i.index(1))

In [19]:
def evaluate(questions, options, correct_option, model):
    embeddings1 = model.encode(questions)
    embeddings2 = model.encode(options)
    
    similarities = []
    for emb1, emb2 in zip(embeddings1, embeddings2):
        similarity = (cosine_similarity([emb1], [emb2])[0][0]+1)/2
        similarities.append(similarity)
        
    
    probs = [similarities[i:i+4] for i in range(0, len(similarities), 4)]
    preds = [inner.index(max(inner)) for inner in probs]
    
    acc = accuracy_score(preds, correct_option)
    
    print("Accuracy Score: ", acc)

In [18]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Zero Shot Evaluation

In [20]:
evaluate(train_questions, train_options, correct_options, model)

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Accuracy Score:  0.4161735700197239


In [21]:
evaluate(eval_questions, eval_options, correct_options_eval, model)

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Accuracy Score:  0.49166666666666664


# Fine Tuning

In [23]:
train_examples = []
val_examples = []
for i in range(len(train_questions)):
    inputEx = InputExample(texts=[train_questions[i], train_options[i]], label=float(train_labels[i]))
    train_examples.append(inputEx)
    
for i in range(len(eval_questions)):
    inputEx = InputExample(texts= [eval_questions[i], eval_options[i]], label=float(eval_labels[i]))
    val_examples.append(inputEx)

In [24]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=1)
val_dataloader = DataLoader(val_examples, shuffle=False, batch_size=1)

In [27]:
train_loss = losses.CosineSimilarityLoss(model)

In [22]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100,
         evaluation_steps = 507, show_progress_bar = True)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2028 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2028 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2028 [00:00<?, ?it/s]

In [30]:
scores = [float(i) for i in eval_labels]
evaluator = evaluation.EmbeddingSimilarityEvaluator(eval_questions, eval_options, scores)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,
    evaluator=evaluator,
    evaluation_steps=len(train_questions)
)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2028 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2028 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2028 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2028 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2028 [00:00<?, ?it/s]

In [39]:
model.save("/kaggle/working/SBERT-MODEL-PRETRAINED")

# Evaluation

In [35]:
evaluate(train_questions, train_options, correct_options, model)

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Accuracy Score:  0.9901380670611439


In [36]:
evaluate(eval_questions, eval_options, correct_options_eval, model)

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Accuracy Score:  0.8083333333333333


# Loading the Pretrained model

In [40]:
pretrained_model = SentenceTransformer("/kaggle/working/SBERT-MODEL-PRETRAINED")

In [41]:
evaluate(train_questions, train_options, correct_options, pretrained_model)

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Accuracy Score:  0.9901380670611439


In [42]:
evaluate(eval_questions, eval_options, correct_options_eval, pretrained_model)

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Accuracy Score:  0.8083333333333333


# Using Torch

In [43]:
torch.save(model.state_dict(), "/kaggle/working/sbert-model.pt")

In [48]:
pt_model = SentenceTransformer("all-MiniLM-L6-v2")
pt_model.load_state_dict(torch.load("/kaggle/working/sbert-model.pt"))

<All keys matched successfully>

In [49]:
evaluate(train_questions, train_options, correct_options, pt_model)

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

Accuracy Score:  0.9901380670611439


In [50]:
evaluate(eval_questions, eval_options, correct_options_eval, pt_model)

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

Accuracy Score:  0.8083333333333333
